In [ ]:
!pip install fastapi uvicorn onnxruntime-openvino pyngrok pillow torchvision


In [ ]:
!pip install pyngrok
!ngrok config add-authtoken 2v3gxXjMShIuTuJJqO976oroL5I_2rtERLHqNan6dtstWMR46
!pip install python-multipart


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import torch
from PIL import Image
from fastapi import FastAPI, File, UploadFile, HTTPException
import onnxruntime as ort
import numpy as np
import io
import torchvision.transforms as transforms
from pyngrok import ngrok
import uvicorn
import threading

# Load ONNX model with OpenCL
providers = ["OpenCLExecutionProvider"]
ort_session = ort.InferenceSession("/content/resnet50_dog_cat.onnx", providers=providers)

# Define class labels
CLASS_NAMES = ["Cat", "Dog"]

app = FastAPI()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(image: Image.Image) -> np.ndarray:
    """Preprocess the image to match model input size"""
    image = transform(image).numpy().astype(np.float32)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        # Read and process the image
        image_bytes = await file.read()
        with io.BytesIO(image_bytes) as image_stream:
            image = Image.open(image_stream).convert("RGB")

        input_tensor = preprocess_image(image)

        # Run inference
        ort_inputs = {ort_session.get_inputs()[0].name: input_tensor}
        raw_outputs = ort_session.run(None, ort_inputs)

        print("Raw Model Output:", raw_outputs[0][0])  # Debugging step

        # Get prediction
        probabilities = raw_outputs[0][0]  # Model outputs probabilities or logits
        predicted_index = np.argmax(probabilities)
        predicted_class = CLASS_NAMES[predicted_index]

        # Ensure probability is between 0 and 1
        confidence_score = float(probabilities[predicted_index])
        confidence_score = min(max(confidence_score, 0), 1)  # Clamp between 0 and 1

        return {
            "prediction": predicted_class,
            "confidence": round(confidence_score, 5)  # Round to 5 decimal places
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing image: {str(e)}")

# Expose FastAPI using ngrok
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Run FastAPI in a separate thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()


Public URL: https://c949-34-145-168-69.ngrok-free.app
